In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S8'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
muIterate = 0.1
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

# Cluster Robot Settings
muStart = 0.60
muEnd = 0.60
muIncr = 0.05
betaStart = 14
betaEnd = 14
betaIncr = 1
sigmaStart = 145
sigmaEnd = 145
sigmaIncr = 1
epsilonStart = 35
epsilonEnd = 35
epsilonIncr = 1
mu = muStart
while mu <= muEnd:
    beta = betaStart
    while beta <= betaEnd:
        sigma = sigmaStart
        while sigma <= sigmaEnd:
            epsilon = epsilonStart
            while epsilon <= epsilonEnd:
                suffixString =','+str(epsilon)+','+str(epsilonIterate)
                resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIterate,epsilon,  \
                                                     epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
                seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
                seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
                seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
                seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
                outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
                resultsFile = open('RobotResults.txt','a+')
                resultsFile.write(outString)
                resultsFile.close()
                epsilon = epsilon + epsilonIncr
            sigma = sigma + sigmaIncr
        beta = beta + betaIncr
    mu = mu + muIncr


 

Tokenizing References
Total References= 1000
Total Tokens Found = 13025
Total Tokens Output = 12511
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 1000
Total Tokens= 12511
Unique Tokens= 3978
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  3
Maximum Frequency of Error Token =  3
References Processed= 1000
Sorted Token Size = 3978
Clean Token Size = 1776
*Stop Replacements here
References Processed =  1000
Total Replacement Pairs = 240
Tokens Read = 12511
Tokens Changed =  256
References Changed = 233

>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 35
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1715
NewMatrix Iterateblocks Total Time = 1.8763923990000002

Precision = TP / (TP + FP) ......................... = 0.78797468
Recall = TP / (TP + FN) ............................ = 0.88801712
F-Measure .......................................... = 0.83501006
